In [2]:
!pip install python-telegram-bot

  Using cached https://files.pythonhosted.org/packages/26/aa/82dfedea526128a55429527ca5e43c023846e00cc3df30c8dbfebe94bbb2/python_telegram_bot-12.2.0-py2.py3-none-any.whl


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This program is dedicated to the public domain under the CC0 license.

"""
First, a few callback functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
Usage:
Example of a bot-user conversation using ConversationHandler.
Send /start to initiate the conversation.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram import ReplyKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

reply_keyboard = [['Age', 'Favourite colour'],
                  ['Number of siblings', 'Something else...'],
                  ['Done']]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)


reply_keyboard_2 = [""]
def facts_to_str(user_data):
    facts = list()

    for key, value in user_data.items():
        facts.append('{} - {}'.format(key, value))

    return "\n".join(facts).join(['\n', '\n'])


def start(update, context):
    update.message.reply_text(
        "Hi! My name is Doctor Botter. I will hold a more complex conversation with you. "
        "Why don't you tell me something about yourself?",
        reply_markup=markup)

    return CHOOSING


def regular_choice(update, context):
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text(
        'Your {}? Yes, I would love to hear about that!'.format(text.lower()))

    return TYPING_REPLY


def custom_choice(update, context):
    update.message.reply_text('Alright, please send me the category first, '
                              'for example "Most impressive skill"')

    return TYPING_CHOICE


def received_information(update, context):
    user_data = context.user_data
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    del user_data['choice']

    update.message.reply_text("Neat! Just so you know, this is what you already told me:"
                              "{} You can tell me more, or change your opinion"
                              " on something.".format(facts_to_str(user_data)),
                              reply_markup=markup)

    return CHOOSING


def done(update, context):
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    update.message.reply_text("I learned these facts about you:"
                              "{}"
                              "Until next time!".format(facts_to_str(user_data)))

    user_data.clear()
    return ConversationHandler.END


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CHOOSING: [MessageHandler(Filters.regex('^(Age|Favourite colour|Number of siblings)$'),
                                      regular_choice),
                       MessageHandler(Filters.regex('^Something else...$'),
                                      custom_choice)
                       ],

            TYPING_CHOICE: [MessageHandler(Filters.text,
                                           regular_choice)
                            ],

            TYPING_REPLY: [MessageHandler(Filters.text,
                                          received_information),
                           ],
        },

        fallbacks=[MessageHandler(Filters.regex('^Done$'), done)]
    )

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

In [ ]:
import logging
import pandas as pd
import re
import time

from telegram import ReplyKeyboardMarkup, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler, CallbackQueryHandler, Handler)


# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

(START, FIND_SPEC, ABOUT, FIND_SPEC_NOW, STOP_SEARCH, SEARCH_SPEC, FIND_SPEC_LATER, SPEC_LIST, GET_INFO, BACK, GENDER, AGE, PROBLEM_CATEGORY, 
 STATEMENT_CATEGORY, FIND_SPEC_LATER_CONFIRMATION, USER_DATA_OK, CHOOSING_ABOUT) = range(17)

# Some constant vars for user data
MALE, FEMALE = "Male", "Female" 

FAMILY, FRIENDS, LOVE, WORK, STUDY, OTHER_PROBLEM = 'family', 'friends', 'love', 'work', 'study', 'other_problem'

ST_CAT_1, ST_CAT_2, ST_CAT_3, ST_CAT_4, ST_CAT_5, ST_CAT_6, ST_CAT_OTHER = 'st_1','st_2','st_3','st_4','st_5','st_6','st_other'


def start(update, context):
#     reply_keyboard = [['Найти специалиста', 'О сервисе']]

#     markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
#     update.message.reply_text(
#         """Привет! Этот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят! 
#         Давай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", 
#         так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
#         reply_markup=markup)
    # Get user that sent /start and log his name
    user = update.effective_user.first_name
    logger.info("User %s goes to find_spec section.", user)
    # Build InlineKeyboard where each button has a displayed text
    # and a string as callback_data
    # The keyboard is a list of button rows, where each row is in turn
    # a list (hence `[[...]]`).
    keyboard = [
        [InlineKeyboardButton('Найти специалиста', callback_data=str(FIND_SPEC)),
         InlineKeyboardButton('О сервисе', callback_data=str(ABOUT))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.effective_chat.send_message(
        text = """Привет!\n\nЭтот бот поможет тебе справиться с депрессией или другими психологическими проблемами, которые тебя беспокоят!\n\nДавай начнем: выбери "Найти специалиста" или узнай о нас больше выбрав "О сервисе", так же ты можешь выбрать быструю помощь и мы найдем тебе ближайшего освободившегося специалиста""",
        reply_markup=reply_markup
    )
    return START


def stop(update, context):
    """Закончить общение с ботом по команде /stop"""
    update.message.reply_text("""Пока!""")
    
    return ConversationHandler.END

def done(update, context):
    update.message.reply_text("""Пока!""")
    user_data = context.user_data
    if 'choice' in user_data:
        del user_data['choice']

    print(user_data)

    user_data.clear()
    return ConversationHandler.END

def find_spec(update, context):
#     r"""найи специалиста выбор из двух альтернатив и переход к след шагу"""
#     reply_keyboard_loc = [['Быстрая помощь','Подобрать', 'Список Специалистов']]

#     markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
#     update.message.reply_text(
#         """Мы можем помочь подобрать Вам специалиста сразу или в зависимости от характера вашей проблемы, 
#         либо вы можете выбрать его самостоятельно из списка специалистов.""",
#         reply_markup=markup_loc)
    user = update.effective_user.first_name
    logger.info("User %s goes to find_spec section.", user)
    #Update message and keyboard
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton('Быстрая помощь', callback_data=str(FIND_SPEC_NOW))],
        [InlineKeyboardButton('Подобрать специалиста', callback_data=str(FIND_SPEC_LATER))],
        [InlineKeyboardButton('Список специалистов', callback_data=str(SPEC_LIST))],
#         [InlineKeyboardButton('Назад', callback_data=str(START))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Мы можем помочь подобрать тебе специалиста сразу или в зависимости от характера твоей проблемы, либо ты можешь выбрать его самостоятельно из списка специалистов.",
        reply_markup=reply_markup
        )
    
    return FIND_SPEC

def find_spec_later(update, context):
    
    #IN PROGRESS
    
    # Section for complex search of specs
    user = update.effective_user.first_name
    logger.info("User %s goes to find_spec_later section.", user) 
    
    query = update.callback_query
    bot = context.bot
    
    keyboard = [
        [InlineKeyboardButton('Хорошо!', callback_data=str(GET_INFO))],
        [InlineKeyboardButton('Назад', callback_data=str(FIND_SPEC))]
#         [InlineKeyboardButton('Назад', callback_data=str(START))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Расскажи о себе, чтобы мы смогли подобрать подходящего специалиста.",
        reply_markup=reply_markup
        )
    if 'user_id' not in context.user_data.keys():
        context.user_data['user_id'] = update.effective_user.id
        
#     if context.user_data['user_id'] == None:
#         context.user_data['user_id'] = update.effective_user.id
#     print(update.message)
#     for data in context.user_data['find_spec_later_data'].columns:
#         if data == 'gender':            
#             print('2')
#             result = gender(update, context)
#             print('3')
#             print(update.callback_query)
#             while update.inline_query == None:
#                 user_data.loc[update.effective_user.id, 'gender'] = update.inline_query.query
#                 print(user_data.loc[update.effective_user.id, 'gender'])
    return GET_INFO

def gender(update, context):
    logger.info("User %s goes to gender section.", update.effective_user.first_name)

    keyboard = [
        [InlineKeyboardButton('Мужской', callback_data = str(MALE)),
         InlineKeyboardButton('Женский', callback_data = str(FEMALE))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.effective_chat.send_message(
        text = "Твой пол?",
        reply_markup=reply_markup
    )
    
#     while (update.callback_query.data != 'Мужской' and update.callback_query.data != 'Женский'):
#         time.sleep(5)
#         print(update.callback_query.data)
#         continue
#     context.user_data['find_spec_later_data'].loc[update.effective_user.id, 'gender'] = update.callback_query.data
#     print(context.user_data['find_spec_later_data'].loc[update.effective_user.id, 'gender'])
    
    return GET_INFO

def spec_list(update, context):
    pass

# def gender(update, context):
#     r"""Пол"""
#     reply_keyboard_loc = [['М', 'Ж']]
#     markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    
#     update.message.reply_text("""Выберите ваш пол.""", reply_markup=markup_loc)
#     return CHOOSE_AGE


def age(update, context):
    r"""Возраст"""
    logger.info("User %s goes to age section.", update.effective_user.first_name)
    context.user_data['gender'] = update.callback_query.data
    
    print(context.user_data)
 
    update.effective_chat.send_message(
        text = "Сколько тебе лет?"
    )
    
    return GET_INFO

def city(update, context):
    r"""Город проживания"""
    logger.info("User %s goes to  city section.", update.effective_user.first_name)
    context.user_data['age'] = update.message.text
    
    print(context.user_data)
 
    update.effective_chat.send_message(
        text = "В каком городе ты живешь?"
    )
    
    return GET_INFO

def problem_category(update, context):
    r"""Выбрать проблему из предложенных типов"""
    logger.info("User %s goes to problem_category section.", update.effective_user.first_name)
    
    context.user_data['city'] = update.message.text
    print(context.user_data)
    
    keyboard = [
        [InlineKeyboardButton('Семья', callback_data = str(FAMILY))],
        [InlineKeyboardButton('Друзья', callback_data = str(FRIENDS))],
        [InlineKeyboardButton('Отношения', callback_data = str(LOVE))],
        [InlineKeyboardButton('Работа', callback_data = str(WORK))],
        [InlineKeyboardButton('Учеба', callback_data = str(STUDY))],
        [InlineKeyboardButton('Другое', callback_data = str(OTHER_PROBLEM))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.effective_chat.send_message(
        text = "К какой категории Вы могли бы отнести свою психологическую проблему?",
        reply_markup=reply_markup
    )
    
    return GET_INFO

def statement_category(update, context):
    r"""Выбрать утверждение из предложенных типов"""
    logger.info("User %s goes to statement_category section.", update.effective_user.first_name)
    
    context.user_data['problem_category'] = update.callback_query.data
    keyboard = [
        [InlineKeyboardButton('Я запутался(лась)', callback_data = str(ST_CAT_1))],
        [InlineKeyboardButton('Мне не интересно или скучно', callback_data = str(ST_CAT_2))],
        [InlineKeyboardButton('Я устал(а)', callback_data = str(ST_CAT_3))],
        [InlineKeyboardButton('Меня никто не слушает', callback_data = str(ST_CAT_4))],
        [InlineKeyboardButton('Не знаю что делать дальше', callback_data = str(ST_CAT_5))],
        [InlineKeyboardButton('Не знаю как сделать правильный выбор', callback_data = str(ST_CAT_6))],
        [InlineKeyboardButton('Другое', callback_data = str(ST_CAT_OTHER))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.effective_chat.send_message(
        text = "Какое утверждение лучше всего характеризует твою проблему?",
        reply_markup=reply_markup
    )    
    
    return GET_INFO

def find_spec_later_confirmation(update, context):
    # Подтверждения правильности данных у пользователя
    logger.info("User %s goes to find_spec_later_confirmation section.", update.effective_user.first_name)
    
    context.user_data['statement_category'] = update.callback_query.data
    keyboard = [
        [InlineKeyboardButton('Все верно', callback_data = str(FIND_SPEC_NOW))],
        [InlineKeyboardButton('Нет, я хочу что-то исправить', callback_data = str(FIND_SPEC_LATER))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.effective_chat.send_message(
        text = "{0}. Все ли верно?".format(context.user_data), # Необходимо сделать красиво для юзера
        reply_markup=reply_markup
    )    
    
    return FIND_SPEC_LATER_CONFIRMATION
    
def find_spec_now(update, context):
    r"""Подбор специалиста сразу"""
    logger.info("User %s goes to  find_spec_now section.", update.effective_user.first_name)
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton('Отменить поиск', callback_data=str(STOP_SEARCH))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Мы уже ищем специалиста. Ожидайте...",
        reply_markup=reply_markup
    )
    
    ''' Нужно допилить алгоритм поиска специалиста'''
    def spec_search(update, context):
        if context.user_data == {}:
            print('Fast search')
        else:
            print('Regular search, data = {0}'.format(context.user_data))
        pass
    
    spec_search(update, context)
    
    return FIND_SPEC_NOW

def stop_search(update, context):
    # stop search of spec
    logger.info("User %s goes to stop_search section.", update.effective_user.first_name)
 
    update.effective_chat.send_message(
        text = "Поиск специалиста отменен."
    )
    find_spec(update, context)
    
    return FIND_SPEC
    
def about(update, context):
    r"""О нас"""
    reply_keyboard_loc = [['О сервисе', 'Стоимость услуг', 'Отзывы', "О нас пишут", "FAQ" ]]
    markup_loc = ReplyKeyboardMarkup(reply_keyboard_loc, one_time_keyboard=True)
    update.message.reply_text("""PsyHelp - бот психологической поддержки""", reply_markup=markup_loc)
    
    return CHOOSING_ABOUT
    
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("834607569:AAFOC-hNLiwAKQ_7I7_pOED9DoTgz1g61Q8", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
#     conv_handler2 = ConversationHandler(
#         entry_points=[CommandHandler('start', start)],

#         states={
#             CHOOSING: [MessageHandler(Filters.regex('^Найти специалиста$'),
#                                       Find_Spec),
#                        MessageHandler(Filters.regex('^О сервисе$'),
#                                       about),
#                        ],
#             STEP_2_FIND_SPEC: [ MessageHandler(Filters.regex('^Быстрая помощь$'),
#                                       find_spec_now),
#                                MessageHandler(Filters.regex('^Подобрать$'),
#                                       Pol),
#                        MessageHandler(Filters.regex('^Список Специалистов$'),
#                                       Pol)
#             ],

#             CHOOSE_AGE: [MessageHandler(Filters.regex('^.*$'), Age) 
#                             ],
            
#             PROBLEM_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Problem_Category)
#                             ],
            
#             QUESTION_CATEGORY: [MessageHandler(Filters.regex('^.*$'), Question_Category)
#                             ],
#         },

#         fallbacks=[MessageHandler(Filters.regex('^Done$'), done),
#                   CommandHandler('stop', stop)]
#     )
    
    find_spec_later_data = ConversationHandler(
        entry_points=
                    [CallbackQueryHandler(find_spec_later, pattern='^' + str(FIND_SPEC_LATER) + '$')],
        states=
        {
            GET_INFO: 
                    [
                    CallbackQueryHandler(gender, pattern='^' + str(GET_INFO) + '$'),
                    CallbackQueryHandler(age, pattern='^' + str(MALE) + '$|^' + str(FEMALE) + '$'),
                    MessageHandler(Filters.regex(r'\d'), city),
                    MessageHandler(Filters.regex(r'[А-я]|[A-z]'), problem_category),
                    CallbackQueryHandler(statement_category, pattern='^' + '|'.join([FAMILY, FRIENDS, LOVE, WORK, STUDY, OTHER_PROBLEM]) + '$'),
                    CallbackQueryHandler(find_spec_later_confirmation, pattern='^' + '|'.join([ST_CAT_1, ST_CAT_2, ST_CAT_3, ST_CAT_4, ST_CAT_5, ST_CAT_6, ST_CAT_OTHER]) + '$')
                    ],
            FIND_SPEC_LATER_CONFIRMATION:
                    [
                    CallbackQueryHandler(find_spec_later, pattern='^' + str(FIND_SPEC_LATER) + '$')
                    ]
        },
        fallbacks=
                    [CommandHandler('start', start),
                    CallbackQueryHandler(start, pattern='^' + str(START) + '$'),
                    CallbackQueryHandler(find_spec, pattern='^' + str(FIND_SPEC) + '$'),
                    CallbackQueryHandler(find_spec_now, pattern='^' + str(FIND_SPEC_NOW) + '$')
                    ],
        map_to_parent=
        {
            # Return to second level menu
            FIND_SPEC: FIND_SPEC,
            FIND_SPEC_NOW: FIND_SPEC_NOW
        }
    )
    
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            START: 
                    [
                    CallbackQueryHandler(find_spec, pattern='^' + str(FIND_SPEC) + '$'),
                    CallbackQueryHandler(about, pattern='^' + str(ABOUT) + '$')
                    ],
            
            FIND_SPEC: 
                    [
                    find_spec_later_data, 
                    CallbackQueryHandler(find_spec_now, pattern='^' + str(FIND_SPEC_NOW) + '$'),
                    CallbackQueryHandler(spec_list, pattern='^' + str(SPEC_LIST) + '$')
                    ],
            
            FIND_SPEC_NOW:   
                    [
                    CallbackQueryHandler(stop_search, pattern='^' + str(STOP_SEARCH) + '$') 
                    ]
#             FIND_SPEC_NOW: [CallbackQueryHandler(start, pattern='^' + str(START) + '$')],
            
#             ABOUT: [CallbackQueryHandler(start_over, pattern='^' + str(ONE) + '$'),
#                      CallbackQueryHandler(end, pattern='^' + str(TWO) + '$')]
        },
        fallbacks=
                    [
                    CommandHandler('start', start),
                    CallbackQueryHandler(start, pattern='^' + str(START) + '$')
                    ]
    )    

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2019-12-12 14:35:03,020 - __main__ - INFO - User Vladimir goes to find_spec section.
2019-12-12 14:35:04,955 - __main__ - INFO - User Vladimir goes to find_spec section.
2019-12-12 14:35:06,302 - __main__ - INFO - User Vladimir goes to find_spec_later section.
2019-12-12 14:35:07,545 - __main__ - INFO - User Vladimir goes to find_spec section.
2019-12-12 14:35:09,281 - __main__ - INFO - User Vladimir goes to find_spec_later section.
2019-12-12 14:35:10,753 - __main__ - INFO - User Vladimir goes to find_spec section.
2019-12-12 14:35:11,990 - __main__ - INFO - User Vladimir goes to find_spec_later section.
2019-12-12 14:35:13,437 - __main__ - INFO - User Vladimir goes to find_spec section.
2019-12-12 14:35:15,074 - __main__ - INFO - User Vladimir goes to  find_spec_now section.


Regular search, data = {'user_id': 259294833}


2019-12-12 14:35:16,509 - __main__ - INFO - User Vladimir goes tostop_search section.
2019-12-12 14:35:16,688 - __main__ - INFO - User Vladimir goes to find_spec section.
2019-12-12 14:35:19,298 - __main__ - INFO - User Vladimir goes to find_spec_later section.
2019-12-12 14:35:20,616 - __main__ - INFO - User Vladimir goes to gender section.
2019-12-12 14:35:27,941 - __main__ - INFO - User Vladimir goes to age section.


{'user_id': 259294833, 'gender': 'Male'}


2019-12-12 14:35:31,124 - __main__ - INFO - User Vladimir goes to  city section.


...

2019-12-12 14:42:44,077 - __main__ - INFO - User Родион goes to  city section.


{'user_id': 11782046, 'gender': 'Male', 'age': '123', 'city': 'кек', 'problem_category': 'other_problem', 'statement_category': 'st_other'}


2019-12-12 14:42:59,579 - __main__ - INFO - User Родион goes to problem_category section.


{'user_id': 11782046, 'gender': 'Male', 'age': '123', 'city': '💩Дерьмовом', 'problem_category': 'other_problem', 'statement_category': 'st_other'}


2019-12-12 14:43:02,542 - __main__ - INFO - User Родион goes to statement_category section.
2019-12-12 14:43:03,322 - __main__ - INFO - User Родион goes to find_spec_later_confirmation section.


In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Simple inline keyboard bot with multiple CallbackQueryHandlers.

This Bot uses the Updater class to handle the bot.
First, a few callback functions are defined as callback query handler. Then, those functions are
passed to the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
Usage:
Example of a bot that uses inline keyboard that has multiple CallbackQueryHandlers arranged in a
ConversationHandler.
Send /start to initiate the conversation.
Press Ctrl-C on the command line to stop the bot.
"""
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler, ConversationHandler
import logging

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

# Stages
FIRST, SECOND = range(2)
# Callback data
ONE, TWO, THREE, FOUR = range(4)


def start(update, context):
    """Send message on `/start`."""
    # Get user that sent /start and log his name
    user = update.message.from_user
    logger.info("User %s started the conversation.", user.first_name)
    # Build InlineKeyboard where each button has a displayed text
    # and a string as callback_data
    # The keyboard is a list of button rows, where each row is in turn
    # a list (hence `[[...]]`).
    keyboard = [
        [InlineKeyboardButton("1", callback_data=str(ONE)),
         InlineKeyboardButton("2", callback_data=str(TWO))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Send message with text and appended InlineKeyboard
    update.message.reply_text(
        "Start handler, Choose a route",
        reply_markup=reply_markup
    )
    # Tell ConversationHandler that we're in state `FIRST` now
    return FIRST


def start_over(update, context):
    """Prompt same text & keyboard as `start` does but not as new message"""
    # Get CallbackQuery from Update
    query = update.callback_query
    # Get Bot from CallbackContext
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton("1", callback_data=str(ONE)),
         InlineKeyboardButton("2", callback_data=str(TWO))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    # Instead of sending a new message, edit the message that
    # originated the CallbackQuery. This gives the feeling of an
    # interactive menu.
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Start handler, Choose a route",
        reply_markup=reply_markup
    )
    return FIRST


def one(update, context):
    """Show new choice of buttons"""
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton("3", callback_data=str(THREE)),
         InlineKeyboardButton("4", callback_data=str(FOUR))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text=("First CallbackQueryHandler, Choose a route" + str(ONE)),
        reply_markup=reply_markup
    )
    return FIRST


def two(update, context):
    """Show new choice of buttons"""
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton("1", callback_data=str(ONE)),
         InlineKeyboardButton("3", callback_data=str(THREE))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Second CallbackQueryHandler, Choose a route",
        reply_markup=reply_markup
    )
    return FIRST


def three(update, context):
    """Show new choice of buttons"""
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton("Yes, let's do it again!", callback_data=str(ONE)),
         InlineKeyboardButton("Nah, I've had enough ...", callback_data=str(TWO))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Third CallbackQueryHandler. Do want to start over?",
        reply_markup=reply_markup
    )
    # Transfer to conversation state `SECOND`
    return SECOND


def four(update, context):
    """Show new choice of buttons"""
    query = update.callback_query
    bot = context.bot
    keyboard = [
        [InlineKeyboardButton("2", callback_data=str(TWO)),
         InlineKeyboardButton("4", callback_data=str(FOUR))]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="Fourth CallbackQueryHandler, Choose a route",
        reply_markup=reply_markup
    )
    return FIRST


def end(update, context):
    """Returns `ConversationHandler.END`, which tells the
    ConversationHandler that the conversation is over"""
    query = update.callback_query
    bot = context.bot
    bot.edit_message_text(
        chat_id=query.message.chat_id,
        message_id=query.message.message_id,
        text="See you next time!"
    )
    return ConversationHandler.END


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Setup conversation handler with the states FIRST and SECOND
    # Use the pattern parameter to pass CallbackQueries with specific
    # data pattern to the corresponding handlers.
    # ^ means "start of line/string"
    # $ means "end of line/string"
    # So ^ABC$ will only allow 'ABC'
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            FIRST: [CallbackQueryHandler(one, pattern='^' + str(ONE) + '$'),
                    CallbackQueryHandler(two, pattern='^' + str(TWO) + '$'),
                    CallbackQueryHandler(three, pattern='^' + str(THREE) + '$'),
                    CallbackQueryHandler(four, pattern='^' + str(FOUR) + '$')],
            SECOND: [CallbackQueryHandler(start_over, pattern='^' + str(ONE) + '$'),
                     CallbackQueryHandler(end, pattern='^' + str(TWO) + '$')]
        },
        fallbacks=[CommandHandler('start', start)]
    )

    # Add ConversationHandler to dispatcher that will be used for handling
    # updates
    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

/Users/vladimirbadlo/anaconda3/envs/psychat/lib/python3.7/site-packages/telegram/ext/conversationhandler.py:215: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn("If 'per_message=False', 'CallbackQueryHandler' will not be "
2019-12-11 18:06:30,493 - telegram.ext.updater - ERROR - Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2019-12-11 18:06:30,494 - __main__ - WARNING - Update "None" caused error "Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
2019-12-11 18:06:36,228 - telegram.ext.updater - ERROR - Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2019-12-11 18:06:36,229 - __main__ - WARNING - Update "None" caused error "Conflict: terminated by other getUpdates request; make sure that only one bot instance is run

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This program is dedicated to the public domain under the CC0 license.

"""
First, a few callback functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Example of a bot-user conversation using nested ConversationHandlers.
Send /start to initiate the conversation.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram import (InlineKeyboardMarkup, InlineKeyboardButton)
from telegram.ext import (Updater, CommandHandler, MessageHandler, Filters,
                          ConversationHandler, CallbackQueryHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

# State definitions for top level conversation
SELECTING_ACTION, ADDING_MEMBER, ADDING_SELF, DESCRIBING_SELF = map(chr, range(4))
# State definitions for second level conversation
SELECTING_LEVEL, SELECTING_GENDER = map(chr, range(4, 6))
# State definitions for descriptions conversation
SELECTING_FEATURE, TYPING = map(chr, range(6, 8))
# Meta states
STOPPING, SHOWING = map(chr, range(8, 10))
# Shortcut for ConversationHandler.END
END = ConversationHandler.END

# Different constants for this example
(PARENTS, CHILDREN, SELF, GENDER, MALE, FEMALE, AGE, NAME, START_OVER, FEATURES,
 CURRENT_FEATURE, CURRENT_LEVEL) = map(chr, range(10, 22))


# Helper
def _name_switcher(level):
    if level == PARENTS:
        return ('Father', 'Mother')
    elif level == CHILDREN:
        return ('Brother', 'Sister')


# Top level conversation callbacks
def start(update, context):
    """Select an action: Adding parent/child or show data."""
    text = 'You may add a familiy member, yourself show the gathered data or end the ' \
           'conversation. To abort, simply type /stop.'
    buttons = [[
        InlineKeyboardButton(text='Add family member', callback_data=str(ADDING_MEMBER)),
        InlineKeyboardButton(text='Add yourself', callback_data=str(ADDING_SELF))
    ], [
        InlineKeyboardButton(text='Show data', callback_data=str(SHOWING)),
        InlineKeyboardButton(text='Done', callback_data=str(END))
    ]]
    keyboard = InlineKeyboardMarkup(buttons)

    # If we're starting over we don't need do send a new message
    if context.user_data.get(START_OVER):
        update.callback_query.edit_message_text(text=text, reply_markup=keyboard)
    else:
        update.message.reply_text('Hi, I\'m FamiliyBot and here to help you gather information'
                                  'about your family.')
        update.message.reply_text(text=text, reply_markup=keyboard)

    context.user_data[START_OVER] = False
    return SELECTING_ACTION


def adding_self(update, context):
    """Add information about youself."""
    context.user_data[CURRENT_LEVEL] = SELF
    text = 'Okay, please tell me about yourself.'
    button = InlineKeyboardButton(text='Add info', callback_data=str(MALE))
    keyboard = InlineKeyboardMarkup.from_button(button)

    update.callback_query.edit_message_text(text=text, reply_markup=keyboard)

    return DESCRIBING_SELF


def show_data(update, context):
    """Pretty print gathered data."""
    def prettyprint(user_data, level):
        people = user_data.get(level)
        if not people:
            return '\nNo information yet.'

        text = ''
        if level == SELF:
            for person in user_data[level]:
                text += '\nName: {0}, Age: {1}'.format(person.get(NAME, '-'), person.get(AGE, '-'))
        else:
            male, female = _name_switcher(level)

            for person in user_data[level]:
                gender = female if person[GENDER] == FEMALE else male
                text += '\n{0}: Name: {1}, Age: {2}'.format(gender, person.get(NAME, '-'),
                                                            person.get(AGE, '-'))
        return text

    ud = context.user_data
    text = 'Yourself:' + prettyprint(ud, SELF)
    text += '\n\nParents:' + prettyprint(ud, PARENTS)
    text += '\n\nChildren:' + prettyprint(ud, CHILDREN)

    buttons = [[
        InlineKeyboardButton(text='Back', callback_data=str(END))
    ]]
    keyboard = InlineKeyboardMarkup(buttons)

    update.callback_query.edit_message_text(text=text, reply_markup=keyboard)
    ud[START_OVER] = True

    return SHOWING


def stop(update, context):
    """End Conversation by command."""
    update.message.reply_text('Okay, bye.')

    return END


def end(update, context):
    """End conversation from InlineKeyboardButton."""
    text = 'See you around!'
    update.callback_query.edit_message_text(text=text)

    return END


# Second level conversation callbacks
def select_level(update, context):
    """Choose to add a parent or a child."""
    text = 'You may add a parent or a child. Also you can show the gathered data or go back.'
    buttons = [[
        InlineKeyboardButton(text='Add parent', callback_data=str(PARENTS)),
        InlineKeyboardButton(text='Add child', callback_data=str(CHILDREN))
    ], [
        InlineKeyboardButton(text='Show data', callback_data=str(SHOWING)),
        InlineKeyboardButton(text='Back', callback_data=str(END))
    ]]
    keyboard = InlineKeyboardMarkup(buttons)
    update.callback_query.edit_message_text(text=text, reply_markup=keyboard)

    return SELECTING_LEVEL


def select_gender(update, context):
    """Choose to add mother or father."""
    level = update.callback_query.data
    context.user_data[CURRENT_LEVEL] = level

    text = 'Please choose, whom to add.'

    male, female = _name_switcher(level)

    buttons = [[
        InlineKeyboardButton(text='Add ' + male, callback_data=str(MALE)),
        InlineKeyboardButton(text='Add ' + female, callback_data=str(FEMALE))
    ], [
        InlineKeyboardButton(text='Show data', callback_data=str(SHOWING)),
        InlineKeyboardButton(text='Back', callback_data=str(END))
    ]]

    keyboard = InlineKeyboardMarkup(buttons)
    update.callback_query.edit_message_text(text=text, reply_markup=keyboard)

    return SELECTING_GENDER


def end_second_level(update, context):
    """Return to top level conversation."""
    context.user_data[START_OVER] = True
    start(update, context)

    return END


# Third level callbacks
def select_feature(update, context):
    """Select a feature to update for the person."""
    buttons = [[
        InlineKeyboardButton(text='Name', callback_data=str(NAME)),
        InlineKeyboardButton(text='Age', callback_data=str(AGE)),
        InlineKeyboardButton(text='Done', callback_data=str(END)),
    ]]
    keyboard = InlineKeyboardMarkup(buttons)

    # If we collect features for a new person, clear the cache and save the gender
    if not context.user_data.get(START_OVER):
        context.user_data[FEATURES] = {GENDER: update.callback_query.data}
        text = 'Please select a feature to update.'
        update.callback_query.edit_message_text(text=text, reply_markup=keyboard)
    # But after we do that, we need to send a new message
    else:
        text = 'Got it! Please select a feature to update.'
        update.message.reply_text(text=text, reply_markup=keyboard)

    context.user_data[START_OVER] = False
    return SELECTING_FEATURE


def ask_for_input(update, context):
    """Prompt user to input data for selected feature."""
    context.user_data[CURRENT_FEATURE] = update.callback_query.data
    text = 'Okay, tell me.'
    update.callback_query.edit_message_text(text=text)

    return TYPING


def save_input(update, context):
    """Save input for feature and return to feature selection."""
    ud = context.user_data
    ud[FEATURES][ud[CURRENT_FEATURE]] = update.message.text

    ud[START_OVER] = True

    return select_feature(update, context)


def end_describing(update, context):
    """End gathering of features and return to parent conversation."""
    ud = context.user_data
    level = ud[CURRENT_LEVEL]
    if not ud.get(level):
        ud[level] = []
    ud[level].append(ud[FEATURES])

    # Print upper level menu
    if level == SELF:
        ud[START_OVER] = True
        start(update, context)
    else:
        select_level(update, context)

    return END


def stop_nested(update, context):
    """Completely end conversation from within nested conversation."""
    update.message.reply_text('Okay, bye.')

    return STOPPING


# Error handler
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("984001690:AAHvJPz7H064B0J9CTJE-243qYMSOVhisPs", use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # Set up third level ConversationHandler (collecting features)
    description_conv = ConversationHandler(
        entry_points=[CallbackQueryHandler(select_feature,
                                           pattern='^' + str(MALE) + '$|^' + str(FEMALE) + '$')],

        states={
            SELECTING_FEATURE: [CallbackQueryHandler(ask_for_input,
                                                     pattern='^(?!' + str(END) + ').*$')],
            TYPING: [MessageHandler(Filters.text, save_input)],
        },

        fallbacks=[
            CallbackQueryHandler(end_describing, pattern='^' + str(END) + '$'),
            CommandHandler('stop', stop_nested)
        ],

        map_to_parent={
            # Return to second level menu
            END: SELECTING_LEVEL,
            # End conversation alltogether
            STOPPING: STOPPING,
        }
    )

    # Set up second level ConversationHandler (adding a person)
    add_member_conv = ConversationHandler(
        entry_points=[CallbackQueryHandler(select_level,
                                           pattern='^' + str(ADDING_MEMBER) + '$')],

        states={
            SELECTING_LEVEL: [CallbackQueryHandler(select_gender,
                                                   pattern='^{0}$|^{1}$'.format(str(PARENTS),
                                                                                str(CHILDREN)))],
            SELECTING_GENDER: [description_conv]
        },

        fallbacks=[
            CallbackQueryHandler(show_data, pattern='^' + str(SHOWING) + '$'),
            CallbackQueryHandler(end_second_level, pattern='^' + str(END) + '$'),
            CommandHandler('stop', stop_nested)
        ],

        map_to_parent={
            # After showing data return to top level menu
            SHOWING: SHOWING,
            # Return to top level menu
            END: SELECTING_ACTION,
            # End conversation alltogether
            STOPPING: END,
        }
    )

    # Set up top level ConversationHandler (selecting action)
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            SHOWING: [CallbackQueryHandler(start, pattern='^' + str(END) + '$')],
            SELECTING_ACTION: [
                add_member_conv,
                CallbackQueryHandler(show_data, pattern='^' + str(SHOWING) + '$'),
                CallbackQueryHandler(adding_self, pattern='^' + str(ADDING_SELF) + '$'),
                CallbackQueryHandler(end, pattern='^' + str(END) + '$'),
            ],
            DESCRIBING_SELF: [description_conv],
        },

        fallbacks=[CommandHandler('stop', stop)],
    )
    # Because the states of the third level conversation map to the ones of the
    # second level conversation, we need to be a bit hacky about that:
    conv_handler.states[SELECTING_LEVEL] = conv_handler.states[SELECTING_ACTION]
    conv_handler.states[STOPPING] = conv_handler.entry_points

    dp.add_handler(conv_handler)

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

/Users/vladimirbadlo/anaconda3/envs/psychat/lib/python3.7/site-packages/telegram/ext/conversationhandler.py:215: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  warnings.warn("If 'per_message=False', 'CallbackQueryHandler' will not be "
2019-12-11 17:47:47,825 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1076)
2019-12-11 17:47:47,828 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1076)"
2019-12-11 17:47:49,320 - telegram.ext.updater - ERROR - Error while getting Updates: urllib3 HTTPError [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1076)
2019-12-11 17:47:49,324 - __main__ - WARNING - Update "None" caused error "urllib3 HTTPError [SSL: CERTIFICATE_VER